In [1]:
import cv2
import os
import torch
import json
import numpy as np
from tqdm.notebook import tqdm

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from mmcv.transforms import Compose
from mmdet.utils import get_test_pipeline_cfg

def read_json(json_path):
    with open(json_path) as f:
        data = json.load(f)
    return data

def preprocess(test_pipeline, image):
    if isinstance(image, np.ndarray):
        # Calling this method across libraries will result
        # in module unregistered error if not prefixed with mmdet.
        test_pipeline[0].type = 'mmdet.LoadImageFromNDArray'
    test_pipeline = Compose(test_pipeline)
    return test_pipeline(dict(img=image))

class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, images_dir, annotations_json_path, transform=None):
        self.transform = transform
        self.images_dir = images_dir
        self.annotations_json = read_json(annotations_json_path)


    def __len__(self):
        return len(self.annotations_json['images'])

    def __getitem__(self, idx):
        image_dict = self.annotations_json['images'][idx]
        image_path = os.path.join(self.images_dir, image_dict['file_name'])
        image_id = image_dict['id']

        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            transformed_images = self.transform(image)
        else:
            transformed_images = image

        return image_id, image_path, transformed_images


# calibrationDataloader = DataLoader(calibrationDataset, batch_size=32, shuffle=True)

In [2]:
import torch
from mmdet.apis import DetInferencer

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize([640, 640]),  # Resize
])

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
CONFIG_PATH = '/teamspace/studios/this_studio/mmdetection/rtmdet_tiny_8xb32-300e_coco.py'
WEIGHTS_PATH = '/teamspace/studios/this_studio/mmdetection/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth'
EVAL_DATASET_SIZE = 5000
CALIBRATION_DATASET_SIZE = 1000
BATCH_SIZE = 64

ROOT_DATASET_DIR = '/teamspace/studios/this_studio/COCO'
IMAGES_DIR = os.path.join(ROOT_DATASET_DIR, 'images')
ANNOTATIONS_JSON_PATH = os.path.join(ROOT_DATASET_DIR, 'annotations/instances_val2017.json')
# ANNOTATIONS_JSON_PATH = "/home/shayaan/Desktop/aimet/my_mmdet/temp.json"

model = DetInferencer(model=CONFIG_PATH, weights=WEIGHTS_PATH, device=DEVICE)
evalDataset = CustomImageDataset(images_dir=IMAGES_DIR, annotations_json_path=ANNOTATIONS_JSON_PATH, transform=transform)
eval_data_loader = DataLoader(evalDataset, batch_size=BATCH_SIZE)

DEVICE

[2024-08-23 08:47:21,083] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.2
 [WARNING]  using untested triton version (2.2.0), only 1.0.0 is known to be compatible
Loads checkpoint by local backend from path: /teamspace/studios/this_studio/mmdetection/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std

08/23 08:47:22 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.


/usr/local/lib/python3.10/dist-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


device(type='cuda', index=0)

In [3]:
from tqdm.notebook import tqdm
import torch

from mmdet.models.utils import samplelist_boxtype2tensor
from mmengine.registry import MODELS
from mmcv.transforms import Compose

test_evaluator = model.cfg.test_evaluator
test_evaluator.type = 'mmdet.evaluation.CocoMetric' 
test_evaluator.dataset_meta = model.model.dataset_meta
test_evaluator.ann_file = ANNOTATIONS_JSON_PATH
test_evaluator = Compose(test_evaluator)

collate_preprocessor = model.preprocess
predict_by_feat = model.model.bbox_head.predict_by_feat
rescale = True

preprocessor = MODELS.build(model.cfg.model.data_preprocessor)
def add_pred_to_datasample(data_samples, results_list):
    for data_sample, pred_instances in zip(data_samples, results_list):
        data_sample.pred_instances = pred_instances
    samplelist_boxtype2tensor(data_samples)
    return data_samples

loading annotations into memory...
Done (t=0.57s)
creating index...
index created!


In [4]:
def pass_calibration_data(model: torch.nn.Module, samples: int):
    data_loader = eval_data_loader
    batch_size = data_loader.batch_size
    model.eval()
    batch_ctr = 0
    with torch.no_grad():
        for image_id, image_path, _ in tqdm(data_loader):
            pre_processed = collate_preprocessor(inputs=image_path, batch_size=batch_size)
            _, data = list(pre_processed)[0]
            data = preprocessor(data, False)
            
            preds = model(data['inputs'].to(DEVICE))

            batch_ctr += 1
            if (batch_ctr * batch_size) > samples:
                break  

In [4]:
from aimet_torch.quantsim import QuantizationSimModel
from aimet_common.defs import QuantScheme
from aimet_torch.model_preparer import prepare_model

def exclude_modules_from_quant(sim, modules_to_ignore):
    name_to_quant_wrapper_dict = {}
    for name, module in sim.model.named_modules():
        name_to_quant_wrapper_dict[name] = module

    quant_wrappers_to_ignore = []
    for name in modules_to_ignore:
        quant_wrapper = name_to_quant_wrapper_dict[name]
        quant_wrappers_to_ignore.append(quant_wrapper)

    sim.exclude_layers_from_quantization(quant_wrappers_to_ignore)

dummy_input = torch.rand(1, 3, 640, 640).to(DEVICE)  # Shape for each ImageNet sample is (3 channels) x (224 height) x (224 width)

BASE_PATH = "/teamspace/studios/this_studio/aimet/exported_models_0.01_2/quant_scheme_W@tf _ A@tf_encodings"
# model = prepare_model(model.model)
model = torch.load(f"{BASE_PATH}/rtm_det.pth", map_location=DEVICE)

quant_sim = QuantizationSimModel(model=model,
                                quant_scheme=QuantScheme.post_training_tf_enhanced,
                                default_param_bw=8,
                                default_output_bw=8,
                                config_file=None,
                                dummy_input=dummy_input)

### if load encodings
# quant_sim.load_encodings(encodings="/teamspace/studios/this_studio/aimet/Examples/torch/quantization/sim_model_excluded_modules/rtm_det_torch.encodings")
# quant_sim.load_encodings(encodings="/teamspace/studios/this_studio/aimet/Examples/torch/quantization/quant_scheme_W@tf / A@tf/rtm_det_torch.encodings")
quant_sim.load_encodings(encodings=f"{BASE_PATH}/rtm_det_torch.encodings")

### else compute encodings
# quant_sim.compute_encodings(pass_calibration_data, 1000)

modules_to_ignore = ['backbone.stage2.1.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_14', 'backbone.stage1.1.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_7', 'backbone.stage2.1.blocks.0.conv2.pointwise_conv.conv', 'backbone.stage3.1.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_21', 'backbone.stage4.2.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_30', 'neck.top_down_blocks.0.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_37', 'neck.top_down_blocks.1.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_44', 'neck.bottom_up_blocks.0.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_51', 'neck.bottom_up_blocks.1.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_58']
exclude_modules_from_quant(quant_sim, modules_to_ignore)



2024-08-23 08:47:28,924 - root - INFO - AIMET


2024-08-23 08:47:39,074 - Quant - INFO - No config file provided, defaulting to config file at /usr/local/lib/python3.10/dist-packages/aimet_common/quantsim_config/default_config.json
2024-08-23 08:47:39,112 - Quant - INFO - Unsupported op type Squeeze
2024-08-23 08:47:39,112 - Quant - INFO - Unsupported op type Mean
2024-08-23 08:47:39,128 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default


In [5]:
name, module = list(quant_sim.model.named_children())[0]

# list(list(list(module.named_children())[0][1].named_children())[0][1].named_children())[0][1].set_mode(QcQuantizeOpMode.ACTIVE)
list(list(list(module.named_children())[0][1].named_children())[0][1].named_children())[0][1]._mode

<QcQuantizeOpMode.ANALYSIS: 2>

In [12]:
import os

os.path.exists(BASE_PATH.replace("encodings", "embedded"))

True

In [13]:
import shutil
import os

save_embedded_dir = BASE_PATH.replace("encodings", "embedded")
shutil.rmtree(save_embedded_dir, ignore_errors=True)
os.makedirs(save_embedded_dir, exist_ok=True)

quant_sim.export(path=save_embedded_dir,
    filename_prefix="rtm_det",
    dummy_input=dummy_input.cuda(),
    use_embedded_encodings=True,
    export_to_torchscript=False,)

2024-08-23 08:46:12,099 - Quant - WARNING - Exporting encodings to yaml will be deprecated in a future release. Ensure that your code can work with the exported files ending in ".encodings" which are saved using json format. For the time being, if yaml export is needed, set aimet_common.utils.SAVE_TO_YAML to True.


/usr/local/lib/python3.10/dist-packages/aimet_torch/qc_quantize_op.py:1240: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  input_tensor = torch.tensor(input_tensor)


In [10]:
module_names = dict(model.named_modules())
modules_to_ignore = ['backbone.stage2.1.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_14', 'backbone.stage1.1.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_7', 'backbone.stage2.1.blocks.0.conv2.pointwise_conv.conv', 'backbone.stage3.1.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_21', 'backbone.stage4.2.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_30', 'neck.top_down_blocks.0.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_37', 'neck.top_down_blocks.1.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_44', 'neck.bottom_up_blocks.0.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_51', 'neck.bottom_up_blocks.1.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_58']

modules_to_ignore

['backbone.stage2.1.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_14',
 'backbone.stage1.1.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_7',
 'backbone.stage2.1.blocks.0.conv2.pointwise_conv.conv',
 'backbone.stage3.1.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_21',
 'backbone.stage4.2.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_30',
 'neck.top_down_blocks.0.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_37',
 'neck.top_down_blocks.1.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_44',
 'neck.bottom_up_blocks.0.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_51',
 'neck.bottom_up_blocks.1.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_58']

In [11]:
def exclude_modules_from_quant(sim, modules_to_ignore):
    name_to_quant_wrapper_dict = {}
    for name, module in sim.model.named_modules():
        name_to_quant_wrapper_dict[name] = module

    quant_wrappers_to_ignore = []
    for name in modules_to_ignore:
        quant_wrapper = name_to_quant_wrapper_dict[name]
        quant_wrappers_to_ignore.append(quant_wrapper)

    sim.exclude_layers_from_quantization(quant_wrappers_to_ignore)

exclude_modules_from_quant(quant_sim, modules_to_ignore)

In [7]:
quant_sim.model

GraphModule(
  (backbone): Module(
    (stem): Module(
      (0): Module(
        (conv): StaticGridQuantWrapper(
          (_module_to_wrap): Conv2d(3, 12, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        )
        (bn): Module(
          (module_batch_norm): StaticGridQuantWrapper(
            (_module_to_wrap): BatchNorm()
          )
        )
        (activate): CustomSiLU(
          (sigmoid): StaticGridQuantWrapper(
            (_module_to_wrap): Sigmoid()
          )
          (mul): StaticGridQuantWrapper(
            (_module_to_wrap): Multiply()
          )
        )
      )
      (1): Module(
        (conv): StaticGridQuantWrapper(
          (_module_to_wrap): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (bn): Module(
          (module_batch_norm_1): StaticGridQuantWrapper(
            (_module_to_wrap): BatchNorm()
          )
        )
        (activate): CustomSiLU(
          (sigmoid): StaticGridQu

In [6]:
from glob import glob
from mmengine.structures import InstanceData

def eval_callback(model, save_dir="./temp"):
    data_loader = eval_data_loader
    new_preds = []
    model.eval()
    with torch.no_grad():
        for image_id, image_path, _ in tqdm(data_loader):
            pre_processed = collate_preprocessor(inputs=image_path, batch_size=BATCH_SIZE)
            _, data = list(pre_processed)[0]
            data = preprocessor(data, False)
            preds = model(data['inputs'].to(DEVICE))
            batch_img_metas = [
            data_samples.metainfo for data_samples in data['data_samples']
            ]
            preds = predict_by_feat(*preds, batch_img_metas=batch_img_metas, rescale=True)
            preds = add_pred_to_datasample(data['data_samples'], preds)
            
            for img_id, pred in zip(image_id, preds):
                pred = pred.pred_instances
                new_pred = InstanceData(metainfo={"img_id": int(img_id)})
                new_pred.bboxes = [np.array(p) for p in pred['bboxes'].cpu()]
                new_pred.labels = pred['labels'].cpu()
                new_pred.scores = pred['scores'].cpu()
                new_preds.append(new_pred)

    eval_results = test_evaluator(new_preds)
    
    os.makedirs(save_dir, exist_ok=True)
    num_file = len(glob(f"{save_dir}/eval_acc_*"))
    with open(f"{save_dir}/eval_acc_{num_file}.json", "w") as f:
        json.dump(eval_results, f, indent=4)
    bbox_map = eval_results['bbox_mAP']
    return bbox_map

In [7]:
save_dir = "/teamspace/studios/this_studio/aimet/eval_stats/rtm_ptq"
eval_callback(quant_sim.model, BASE_PATH)

  0%|          | 0/79 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


08/23 09:05:08 - mmengine - INFO - Evaluating bbox...
Loading and preparing results...
DONE (t=3.68s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=76.85s).
Accumulating evaluation results...
DONE (t=21.57s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.385
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.551
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.418
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.202
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.427
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.550
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.321
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.536
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.589
 Average Rec

0.385

In [1]:
BASE_PATH

NameError: name 'BASE_PATH' is not defined